# Setup and Preprocessing

In [2]:
import numpy as np
import pandas as pd
import hotspot
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
import mplscience

from scipy.io import mmread
from scipy.sparse import csr_matrix

import scanpy as sc

In [3]:
counts_raw = mmread("./../data/GSM3302829_embryo3_SeuratNorm.mtx.gz")
counts_raw = csr_matrix(counts_raw)
barcodes = pd.read_table("./../data/GSM3302829_embryo3_SeuratBarcodes.tsv.gz", header=None)[0]
barcodes = [x+'-1' for x in barcodes] # to match the newick file
genes = pd.read_table("./../data/GSM3302829_embryo3_SeuratGenes.tsv.gz", header=None)[0]

In [6]:
# Load the tree and enumerate the leaves
from ete3 import Tree
tree = Tree("./../data/0726_E2-2_tree_greedy_priors.processed.txt", format=1)
leaves = set()
for tn in tree.traverse('postorder'):
    if tn.is_leaf():
        leaves.add(tn.name)
len(leaves)

1756

In [7]:
# Subset the count matrix to only the cells where the lineage was recoverable
is_valid = [x in leaves for x in barcodes]
is_valid_indices = np.nonzero(is_valid)[0]
valid_barcodes = [barcodes[i] for i in is_valid_indices]

In [8]:
adata = sc.AnnData(counts_raw.transpose())
adata.obs_names = barcodes
adata.var_names = genes

adata = adata[valid_barcodes]
sc.pp.filter_genes(adata, min_cells=10)

/tmp/ipykernel_59536/566723758.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = sc.AnnData(counts_raw.transpose())
/home/dennis/miniconda3/envs/bioinformatics/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:251: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number


# Using HotSpot to get the pairwise relationships

In [9]:
# Create the Hotspot object and the neighborhood graph

hs = hotspot.Hotspot(adata, model='normal', tree=tree)

# TODO: Replace this with a TE-based calculation?
hs.create_knn_graph(
    weighted_graph=False, n_neighbors=30,
)

100%|██████████| 1756/1756 [00:11<00:00, 156.16it/s]


In [11]:
hs_results = hs.compute_autocorrelations(jobs=20)

100%|██████████| 12440/12440 [00:16<00:00, 751.61it/s]


,C,Z,Pval,FDR
Gene,,,,
Rhox9,0.397749,67.377366,0.000000e+00,0.000000e+00
Gjb3,0.324565,58.648372,0.000000e+00,0.000000e+00
Gm9,0.299760,52.619236,0.000000e+00,0.000000e+00
Utf1,0.305452,51.477900,0.000000e+00,0.000000e+00
Ldoc1,0.292335,51.123571,0.000000e+00,0.000000e+00
Rhox6,0.281248,49.578977,0.000000e+00,0.000000e+00
Tfap2c,0.275263,45.350145,0.000000e+00,0.000000e+00
Pramef12,0.234209,43.074083,0.000000e+00,0.000000e+00
S100g,0.222336,39.798370,0.000000e+00,0.000000e+00


In [ ]:
hs_results.head(15)

# Extracting the Edges of the Hypergraph

In [ ]:
#TODO and doesn't make sense for HotSpot gives you.